<h1>WayFair Competition Jupyter Notebook Submission</h1>

Author: Alec James

<h3>Data Analysis Pipeline:</h3>

<b>Classes</b>
- [X] ExploratoryAnalysis
- [ ] Preprocessing
- [ ] Cleaning
- [ ] Modeling
- [ ] Visualization
- [ ] Insights




In [2]:
# Standard libraries
import pandas as pd  # Dataframes
import numpy as np  # arrays
import matplotlib.pyplot as plt  # plotting visuals

<h2>Class ExploratoryAnalysis</h2>

The purpose of this class is to hold all my data preprocessing needs such as: importing, cleaning, finding useful statistics, basic visualization.

<h3>Functions and brief descriptions</h3>

- [ ] \__init__(self)
- [ ] logit(self, logString)
- [ ] getDf(self)
- [ ] getLog(self)
- [ ] printLog(self)
- [ ] printList(self)


- [ ] dataImport(self, filename)


- [ ] categoryFeatures(self)
- [ ] categoryTotals(self, columnList)
- [ ] categoryFeaturePositiveComparisons(self, columnList, *compareColumns)
- [ ] categoryStats(self, categoryTotals, compareColumn)




In [7]:
class ExploratoryAnalysis():
    
    '''
    Initiation, log, get, and functions
    '''
    def __init__(self):
        self.df = -1  # main dataframe
        self.log = []  # class log list
        self.logCount = 0  # used for debugging
        
    # Log class actions
    def logit(self, logString):
        s = "LOG:" + str(self.logCount) + " "
        s += (logString)
        self.log.append(s)
        self.logCount += 1   
    
    # Get global DataFrame
    def getDf(self):
        return self.df
    
    # Get global log
    def getLog(self):
        return self.log
    
    def printLog(self):
        self.printList(self.log)
    
    # Print a list object with x amount of columns, default=1
    def printList(self, objList):
        for i, each in enumerate(objList):
            print(each)
            
            
    '''
    Import 
    '''
    # Imports file into a pandas dataframe
    def dataImport(self, filename):
        # Class log block
        if (type(self.df) == pd.DataFrame): 
            self.logit("dataImport- Overwritting class data")
        else: 
            self.logit("dataImport- Importing file")
        
        # Pandas csv data import
        try:
            self.df = pd.read_csv(filename, index_col=0)
        except:
            pass    
        return self.df
    
    
    '''
    Basic statistics functions about data    
    '''
    # Returns the names of all columns with categorical data 
    def categoryFeatures(self):
        categoryColumns = []
        for cat in range(self.df.shape[1]):
            if self.df.dtypes[cat] == 'object':  #object dtypes are categorical
                categoryColumns.append(self.df.columns[cat])
        
        self.logit("CategoryFeatures- " + str(categoryColumns))  # Log
        return(categoryColumns)
    
    # Totals the number of unique categories within each column and returns a count of how many in each category
    def categoryTotals(self, columnList):
        categoryTotals = []
        
        for i, category in enumerate(columnList):
            categoryTotalsDict = {}
            
            for u, uniqueCategoryInCol in enumerate(self.df[category].unique()):
                categoryTotalsDict[uniqueCategoryInCol] = self.df[category].value_counts()[uniqueCategoryInCol]
            
            categoryTotals.append(category)    
            categoryTotals.append(categoryTotalsDict) 
        
        self.logit("categoryTotals- ")
        return categoryTotals
    
    # Cross tabs between each of in the columnList with sum of each from *compareColumns
    def categoryFeaturePositiveComparisons(self, columnList, *compareColumns):
        assert(len(compareColumns) > 0)  
        compareColumnsListForEach = [] 
        
        for column in compareColumns: 
            assert(self.df.dtypes[column] != "object")  # must be a numbered type
            compareColumnsListForEach.append([])

        for category in columnList:
            others = []
            for each in compareColumns:
                others.append({})
                
            for uniqueCategoryInCol in self.df[category].unique():
                for i, cmpCol in enumerate(compareColumns):
                    tempDfBool = (self.df[cmpCol]>0) & (self.df[category] == uniqueCategoryInCol) # boolean df of all the positives
                    tempDfCats = self.df[tempDfBool] # df of just positives 
                    others[i][uniqueCategoryInCol] = tempDfCats[cmpCol].sum().round(2) # sum of the column row
            
            for i, each in enumerate(compareColumns): 
                compareColumnsListForEach[i].append(category)
                compareColumnsListForEach[i].append(others[i])
        self.logit("categoryFeaturePositiveComparisons- ")
        return compareColumnsListForEach
            
    # Function that takes the contents of two lists of dictionaries and divides compareColumn by categoryTotals
    def categoryStats(self, categoryTotals, compareColumn):
        average = []
        for totals, item in zip(categoryTotals, compareColumn):
            averageDict = {}            
            if type(item) == dict:
                for x in item:
                    if(item.get(x) == 0):
                        averageDict[x] = 0
                    else:
                        averageDict[x] = item.get(x) / totals.get(x)
                average.append(averageDict)
        self.logit("categoryStats- ")
        return average

        
        
exp = ExploratoryAnalysis()
exp.dataImport('Training and Holdout Data/small.csv')

categoryColumns = exp.categoryFeatures()
categoryTotals = exp.categoryTotals(categoryColumns)
compareColumnsListForEach = exp.categoryFeaturePositiveComparisons(categoryColumns, 'convert_30', 'revenue_30')

percentLikelihood = exp.categoryStats(categoryTotals, compareColumnsListForEach[0])
averagePerPopulation = exp.categoryStats(categoryTotals, compareColumnsListForEach[1])
averagePerBuyer = exp.categoryStats(compareColumnsListForEach[0], compareColumnsListForEach[1])


print("----Categorical Columns----")
exp.printList(categoryColumns)

print("\n----Categorical Total for each Categorical Column----")
exp.printList(categoryTotals)

print("\n----Buyers Count in each unique category----")
exp.printList(compareColumnsListForEach[0])

print("\n----Buyers Revenue in each unique category----")
exp.printList(compareColumnsListForEach[1])

print("\n----Percent Likelihood someone in that category will buy----")
exp.printList(percentLikelihood)

print("\n----For the people who bought in that category, Average Spending----")
exp.printList(averagePerBuyer)

print("\n----For the entire population in that category, Average Spending of population (including those who did not buy)----")
exp.printList(averagePerPopulation)

----Categorical Columns----
roll_up
currentstatus
companytypegroup
team
customersource
accrole
num_employees
num_purchases_year
cost_purchases_year
enrollmentmethod

----Categorical Total for each Categorical Column----
roll_up
{'Onboarding': 3374}
currentstatus
{'Active': 1133, 'Enrolled': 2240, 'In Progress': 1}
companytypegroup
{'Business': 2711, 'Trade': 663}
team
{'US': 3239, 'CA': 135}
customersource
{'External Application': 418, 'Internal Application': 1446, 'Internal Customer Scrape': 624, 'Gateway': 3, 'Social - Paid': 404, 'Search - Paid': 209, 'Other': 64, 'Quotes': 5, 'Affiliates': 184, 'Referral': 4, 'Bulk Upload': 8, 'Email': 4, 'Display - Retargeting': 1}
accrole
{'None': 1897, 'Primary': 1245, 'Purchaser': 226, 'Other': 6}
num_employees
{'50plus': 105, '2to5': 817, '1': 444, '6to10': 233, 'None': 1585, '11to50': 190}
num_purchases_year
{'1to2': 230, '25plus': 309, '3to5': 437, '11to25': 207, 'None': 1778, '6to10': 413}
cost_purchases_year
{'lessthan1': 228, '25to100': 1